In [ ]:
!pip install datasets transformers==4.18.0 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalli

In [ ]:
from datasets import *
import transformers
from tokenizers import *
import os
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#local dataset
dataset = load_dataset("text",data_files="/content/drive/MyDrive/BERT MODEL/DATASET/uncased_july15.txt",split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# split the dataset into training (90%) and testing (10%)
d = dataset.train_test_split(test_size=0.1)
d["train"], d["test"]

(Dataset({
     features: ['text'],
     num_rows: 253539
 }),
 Dataset({
     features: ['text'],
     num_rows: 28172
 }))

In [ ]:
for t in d["train"]["text"][:3]:
  print(t)
  print("="*50)

"inihayag ng pamunuan ng light rail manila corp lrmc, na namamahala lrt1, na kailangan nilang magbawas ng 100 tauhan dahil na rin paghina ng bunga ng pagbaba ng bilang ng sumasakay tren sanhi ng covid19 pandemic', 'kaniadtong martes, gipahibalo lider light rail manila corp lrmc nagdumala lrt1, kinahanglan magtanggal og usa ka gatos trabahante usab paghinay pagubos kadaghanon nagasakay trn pandemiya covid19"
"na nasa lupain ng canaan, anak ni ruben, at anak ni gad, at kalahating lipi ni manases ay nagtayo roon ng dambana tabi ng jordan, isang malaking dambana na matatanaw, 'ug paghiadto dapit haduol jordan maoy yuta canaan, anak ni ruben, anak ni gad, katungangabanay ni manases, nanagtukod usa ka halaran haduol jordan, usa ka dakung halaran pagatanawon"
"ug ang tanang nagapuyo sa yuta magasimba kaniya, tanan sila kinsang mga ngalan, sa wala pa ang pagkatukod sa kalibutan, wala mahisulat sa basahon sa kinabuhi nga iya sa gipatay nga cordero."


In [ ]:
# if you want to train the tokenizer from scratch (especially if you have custom
# dataset loaded as datasets object), then run this cell to save it as files
# but if you already have your custom data as text files, there is no point using this
def dataset_to_text(dataset, output_filename="data.txt"):
  """Utility function to save dataset text to disk,
  useful for using the texts to train the tokenizer
  (as the tokenizer accepts files)"""
  with open(output_filename, "w") as f:
    for t in dataset["text"]:
      print(t, file=f)

# save the training set to train.txt
dataset_to_text(d["train"], "train.txt")
# save the testing set to test.txt
dataset_to_text(d["test"], "test.txt")

In [ ]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]
# if you want to train the tokenizer on both sets
# files = ["train.txt", "test.txt"]
# training the tokenizer on the training set
files = ["train.txt"]
# 30,522 vocab is BERT's default vocab size, feel free to tweak
vocab_size = 30_000
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 256
# whether to truncate
truncate_longer_samples = True

In [ ]:
# initialize the WordPiece tokenizer
tokenizer = BertWordPieceTokenizer()
# train the tokenizer
tokenizer.train(files=files, vocab_size=vocab_size, special_tokens=special_tokens)
# enable truncation up to the maximum 512 tokens
tokenizer.enable_truncation(max_length=max_length)

In [ ]:
model_path = "/content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/model"
# make the directory if not already there
if not os.path.isdir(model_path):
  os.mkdir(model_path)

In [ ]:
# save the tokenizer
tokenizer.save_model(model_path)

['/content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/model/vocab.txt']

In [ ]:
# dumping some of the tokenizer config to config file,
# including special tokens, whether to lower case and the maximum sequence length
with open(os.path.join(model_path, "config.json"), "w") as f:
  tokenizer_cfg = {
      "do_lower_case": True,
      "unk_token": "[UNK]",
      "sep_token": "[SEP]",
      "pad_token": "[PAD]",
      "cls_token": "[CLS]",
      "mask_token": "[MASK]",
      "model_max_length": max_length,
      "max_len": max_length,
  }
  json.dump(tokenizer_cfg, f)

In [ ]:
!pip install transformers

In [ ]:
from transformers import DistilBertTokenizer


In [ ]:
# when the tokenizer is trained and configured, load it as BertTokenizerFast
tokenizer = DistilBertTokenizer.from_pretrained(model_path)

In [ ]:
type(tokenizer)

transformers.models.distilbert.tokenization_distilbert.DistilBertTokenizer

Testing the tokenizer

In [ ]:
example = "Ipaathag kon paano ginbag-o sang Industrial Revolution ang kalibutan."
encoding = tokenizer(example)

In [ ]:
encoding

{'input_ids': [2, 15616, 393, 6520, 482, 15493, 19, 59, 675, 19592, 8441, 177, 1141, 20, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/model/saved_model')

('/content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/model/saved_model/tokenizer_config.json',
 '/content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/model/saved_model/special_tokens_map.json',
 '/content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/model/saved_model/vocab.txt',
 '/content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/model/saved_model/added_tokens.json')

In [ ]:
def encode_with_truncation(examples):
  """Mapping function to tokenize the sentences passed with truncation"""
  return tokenizer(examples["text"], truncation=True, padding="max_length",
                   max_length=max_length, return_special_tokens_mask=True)

def encode_without_truncation(examples):
  """Mapping function to tokenize the sentences passed without truncation"""
  return tokenizer(examples["text"], return_special_tokens_mask=True)

# the encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation

# tokenizing the train dataset
train_dataset = d["train"].map(encode, batched=True)
# tokenizing the testing dataset
test_dataset = d["test"].map(encode, batched=True)

if truncate_longer_samples:
  # remove other columns and set input_ids and attention_mask as PyTorch tensors
  train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
  test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
  # remove other columns, and remain them as Python lists
  test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
  train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

Map:   0%|          | 0/253539 [00:00<?, ? examples/s]

Map:   0%|          | 0/28172 [00:00<?, ? examples/s]

In [ ]:
len(train_dataset)

253539

In [ ]:
train_dataset[0]

{'input_ids': tensor([    2,     8, 13963,   210, 10656,   210,  8755, 12950,  1458,  5312,
         17422, 10340,    18,   192, 18135, 23328,   136,    18,   192,  5343,
          2021,   760,   512,   210,  2356, 12462,  1036,   192,  1471,  5961,
           115,   210,  2043,   210, 21057,   210,   993,   210, 26617,  5632,
          9589,   210,  1177,  5405,    13,    18,    13,   744,  3436,    18,
          6064,  3777,  8755, 12950,  1458,  5312, 17422, 10340,  7818, 23328,
           136,    18,   614, 28745,  2372,   262,   204,   170,  9278,  5356,
           364, 26592,  8395,   778,   118, 22062, 18701, 22317,   454,   116,
         11617,  1177,     8,     3,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [ ]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
# grabbed from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

if not truncate_longer_samples:
  train_dataset = train_dataset.map(group_texts, batched=True,
                                    desc=f"Grouping texts in chunks of {max_length}")
  test_dataset = test_dataset.map(group_texts, batched=True,
                                  desc=f"Grouping texts in chunks of {max_length}")
  # convert them from lists to torch tensors
  train_dataset.set_format("torch")
  test_dataset.set_format("torch")

In [ ]:
len(train_dataset), len(test_dataset)

(253539, 28172)

In [ ]:
from transformers import DistilBertForMaskedLM, DistilBertConfig,BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments, pipeline

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.2 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
%env WANDB_PROJECT=CBERT_ONE

env: WANDB_PROJECT=CBERT_ONE


In [ ]:
# initialize the model with the config
model_config = DistilBertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
model = DistilBertForMaskedLM(config=model_config)

In [ ]:
# initialize the data collator, randomly masking 20% (default is 15%) of the tokens for the Masked Language
# Modeling (MLM) task
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)

In [ ]:
training_args = TrainingArguments(
    report_to="wandb",
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,
    num_train_epochs=3,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=10, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,  # evaluation batch size
    logging_steps=500,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices


View training arguments for verification

In [ ]:
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=500,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=8,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
log

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2, return_tensors='pt')

In [ ]:
data_collator

DataCollatorForLanguageModeling(tokenizer=PreTrainedTokenizer(name_or_path='/content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/model', vocab_size=30000, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), mlm=True, mlm_probability=0.2, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [ ]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer

In [ ]:
%env WANDB_LOG_MODEL="checkpoint"

env: WANDB_LOG_MODEL="checkpoint"


In [ ]:
#have a look at our model config
model_config

DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 256,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "transformers_version": "4.18.0",
  "vocab_size": 30000
}

In [ ]:
#have a look at our model
model

DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(256, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.

Data Collator.

Training the Model based on the parameters set.

In [ ]:
# train the model
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: text, special_tokens_mask. If text, special_tokens_mask are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 253539
  Num Epochs = 3
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 80
  Gradient Accumulation steps = 8
  Total optimization steps = 9507
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
500,7.006000,6.465328
1000,6.293700,6.038574
1500,5.913100,5.681075
2000,5.627700,5.411565
2500,5.385400,5.230437
3000,5.221500,5.073392
3500,5.064600,4.947770
4000,4.973200,4.847064


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: text, special_tokens_mask. If text, special_tokens_mask are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 28172
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: text, special_tokens_mask. If text, special_tokens_mask are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 28172
  Batch size = 64
Saving model checkpoint to /content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/model/checkpoint-1000
Configuration saved in /content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/model/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/BERT

Step,Training Loss,Validation Loss
500,7.006000,6.465328
1000,6.293700,6.038574
1500,5.913100,5.681075
2000,5.627700,5.411565
2500,5.385400,5.230437
3000,5.221500,5.073392
3500,5.064600,4.947770
4000,4.973200,4.847064
4500,4.877800,4.783420
5000,4.804700,4.696238


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: text, special_tokens_mask. If text, special_tokens_mask are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 28172
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: text, special_tokens_mask. If text, special_tokens_mask are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 28172
  Batch size = 64
Saving model checkpoint to /content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/model/checkpoint-5000
Configuration saved in /content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/model/checkpoint-5000/config.json
Model weights saved in /content/drive/MyDrive/BERT

TrainOutput(global_step=9507, training_loss=5.071243907470639, metrics={'train_runtime': 14024.1534, 'train_samples_per_second': 54.236, 'train_steps_per_second': 0.678, 'total_flos': 5.041226526076109e+16, 'train_loss': 5.071243907470639, 'epoch': 3.0})

In [ ]:
wandb.finish()

eval/loss,█▇▅▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁
eval/runtime,██▁▁▁▁██▁▁█▁████▁▁▂
eval/samples_per_second,▁▁████▁▁██▁█▁▁▁▁██▇
eval/steps_per_second,▁▁████▁▁██▁█▁▁▁▁██▇
train/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇█████
train/global_step,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇█████
train/learning_rate,██▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▁▁
train/loss,█▆▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [ ]:
trainer.save_model('/content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/distilBERT')

Saving model checkpoint to /content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/distilBERT
Configuration saved in /content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/distilBERT/config.json
Model weights saved in /content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/distilBERT/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/distilBERT
Configuration saved in /content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/distilBERT/config.json
Model weights saved in /content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/distilBERT/pytorch_model.bin


In [ ]:
from transformers import DistilBertTokenizer

In [ ]:
# when you load from pretrained
model = DistilBertForMaskedLM.from_pretrained('/content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/distilBERT')
#tokenizer = BertTokenizerFast.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
# or simply use pipeline
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

loading configuration file /content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/distilBERT/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 256,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "vocab_size": 30000
}

loading weights file /content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/distilBERT/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForMaskedLM.

All the weights of DistilBertForMaskedLM were initialized from the model checkpoint at /content/drive/MyDrive/BERT MODEL/DistilBERT_FINAL/uncased_model/distilBERT.
If your task

In [ ]:
# perform predictions
fill_mask(f'Kinsa kabalo mo {fill_mask.tokenizer.mask_token}.')

[{'score': 0.13207867741584778,
  'token': 308,
  'token_str': 'k i n i',
  'sequence': 'kinsa kabalo mo kini.'},
 {'score': 0.08122830092906952,
  'token': 934,
  'token_str': 'n i m o',
  'sequence': 'kinsa kabalo mo nimo.'},
 {'score': 0.04169512167572975,
  'token': 269,
  'token_str': 's i y a',
  'sequence': 'kinsa kabalo mo siya.'},
 {'score': 0.027872053906321526,
  'token': 170,
  'token_str': 'k a',
  'sequence': 'kinsa kabalo mo ka.'},
 {'score': 0.02477985993027687,
  'token': 355,
  'token_str': 'a k o',
  'sequence': 'kinsa kabalo mo ako.'}]

In [ ]:
fill_mask(f'gituyo niya mosunod {fill_mask.tokenizer.mask_token}.')

[{'score': 0.49487438797950745,
  'token': 176,
  'token_str': 'n g a',
  'sequence': 'gituyo niya mosunod nga.'},
 {'score': 0.024267029017210007,
  'token': 457,
  'token_str': 's e n t e n c e',
  'sequence': 'gituyo niya mosunod sentence.'},
 {'score': 0.014190644025802612,
  'token': 308,
  'token_str': 'k i n i',
  'sequence': 'gituyo niya mosunod kini.'},
 {'score': 0.013600281439721584,
  'token': 269,
  'token_str': 's i y a',
  'sequence': 'gituyo niya mosunod siya.'},
 {'score': 0.008054196834564209,
  'token': 446,
  'token_str': 'p u l o n g',
  'sequence': 'gituyo niya mosunod pulong.'}]

In [ ]:
fill_mask(f'isulod nako ni sa imong {fill_mask.tokenizer.mask_token}.')

[{'score': 0.03057894855737686,
  'token': 446,
  'token_str': 'p u l o n g',
  'sequence': 'isulod nako ni sa imong pulong.'},
 {'score': 0.023940756916999817,
  'token': 892,
  'token_str': 'k a u g a l i n g o n',
  'sequence': 'isulod nako ni sa imong kaugalingon.'},
 {'score': 0.02178204618394375,
  'token': 947,
  'token_str': 'k i n a b u h i',
  'sequence': 'isulod nako ni sa imong kinabuhi.'},
 {'score': 0.01725897379219532,
  'token': 713,
  'token_str': 'n g a l a n',
  'sequence': 'isulod nako ni sa imong ngalan.'},
 {'score': 0.011429117992520332,
  'token': 628,
  'token_str': 'a m a h a n',
  'sequence': 'isulod nako ni sa imong amahan.'}]

In [ ]:
mask = fill_mask.tokenizer.mask_token

In [ ]:
fill_mask(f'isulod nako ni sa {mask}')

[{'score': 0.013821197673678398,
  'token': 468,
  'token_str': 'b a l a y',
  'sequence': 'isulod nako ni sa balay'},
 {'score': 0.012323757633566856,
  'token': 770,
  'token_str': 'b a t a',
  'sequence': 'isulod nako ni sa bata'},
 {'score': 0.012149686925113201,
  'token': 947,
  'token_str': 'k i n a b u h i',
  'sequence': 'isulod nako ni sa kinabuhi'},
 {'score': 0.009886234998703003,
  'token': 1141,
  'token_str': 'k a l i b u t a n',
  'sequence': 'isulod nako ni sa kalibutan'},
 {'score': 0.009017960168421268,
  'token': 1924,
  'token_str': 'l i b r o',
  'sequence': 'isulod nako ni sa libro'}]

In [ ]:
fill_mask(f'gikinahanglan ang politika ug {mask}.')

[{'score': 0.027570301666855812,
  'token': 457,
  'token_str': 's e n t e n c e',
  'sequence': 'gikinahanglan ang politika ug sentence.'},
 {'score': 0.0068985531106591225,
  'token': 1284,
  'token_str': 'n u m e r o',
  'sequence': 'gikinahanglan ang politika ug numero.'},
 {'score': 0.006486860569566488,
  'token': 1873,
  'token_str': 'd a t o s',
  'sequence': 'gikinahanglan ang politika ug datos.'},
 {'score': 0.006241977214813232,
  'token': 27,
  'token_str': '5',
  'sequence': 'gikinahanglan ang politika ug 5.'},
 {'score': 0.0055777146480977535,
  'token': 25,
  'token_str': '3',
  'sequence': 'gikinahanglan ang politika ug 3.'}]

In [ ]:
fill_mask(f'Kini adunay {mask} ug mahabog nga paril')

[{'score': 0.006957870442420244,
  'token': 204,
  'token_str': 'u s a',
  'sequence': 'kini adunay usa ug mahabog nga paril'},
 {'score': 0.006138226483017206,
  'token': 334,
  'token_str': 't a w o',
  'sequence': 'kini adunay tawo ug mahabog nga paril'},
 {'score': 0.0053618173114955425,
  'token': 893,
  'token_str': 'p a a g i',
  'sequence': 'kini adunay paagi ug mahabog nga paril'},
 {'score': 0.004489441402256489,
  'token': 1141,
  'token_str': 'k a l i b u t a n',
  'sequence': 'kini adunay kalibutan ug mahabog nga paril'},
 {'score': 0.004450521431863308,
  'token': 615,
  'token_str': 't u i g',
  'sequence': 'kini adunay tuig ug mahabog nga paril'}]

In [ ]:
fill_mask(f'mao kini atong {mask}.')

[{'score': 0.02187158167362213,
  'token': 947,
  'token_str': 'k i n a b u h i',
  'sequence': 'mao kini atong kinabuhi.'},
 {'score': 0.01590564101934433,
  'token': 334,
  'token_str': 't a w o',
  'sequence': 'mao kini atong tawo.'},
 {'score': 0.015655873343348503,
  'token': 713,
  'token_str': 'n g a l a n',
  'sequence': 'mao kini atong ngalan.'},
 {'score': 0.012047935277223587,
  'token': 332,
  'token_str': 'd i o s',
  'sequence': 'mao kini atong dios.'},
 {'score': 0.011902854777872562,
  'token': 547,
  'token_str': 'b u t a n g',
  'sequence': 'mao kini atong butang.'}]

In [ ]:
fill_mask(f'gibuhat nato ni para sa atong {mask}')

[{'score': 0.021120714023709297,
  'token': 468,
  'token_str': 'b a l a y',
  'sequence': 'gibuhat nato ni para sa atong balay'},
 {'score': 0.01722663827240467,
  'token': 892,
  'token_str': 'k a u g a l i n g o n',
  'sequence': 'gibuhat nato ni para sa atong kaugalingon'},
 {'score': 0.015681160613894463,
  'token': 628,
  'token_str': 'a m a h a n',
  'sequence': 'gibuhat nato ni para sa atong amahan'},
 {'score': 0.01428880263119936,
  'token': 947,
  'token_str': 'k i n a b u h i',
  'sequence': 'gibuhat nato ni para sa atong kinabuhi'},
 {'score': 0.012533009983599186,
  'token': 770,
  'token_str': 'b a t a',
  'sequence': 'gibuhat nato ni para sa atong bata'}]

In [ ]:
fill_mask(f'para ni sa atong {mask}.')

[{'score': 0.019216202199459076,
  'token': 947,
  'token_str': 'k i n a b u h i',
  'sequence': 'para ni sa atong kinabuhi.'},
 {'score': 0.016643516719341278,
  'token': 334,
  'token_str': 't a w o',
  'sequence': 'para ni sa atong tawo.'},
 {'score': 0.016280390322208405,
  'token': 892,
  'token_str': 'k a u g a l i n g o n',
  'sequence': 'para ni sa atong kaugalingon.'},
 {'score': 0.01200205460190773,
  'token': 547,
  'token_str': 'b u t a n g',
  'sequence': 'para ni sa atong butang.'},
 {'score': 0.011071482673287392,
  'token': 713,
  'token_str': 'n g a l a n',
  'sequence': 'para ni sa atong ngalan.'}]